# Week 10 Homework

>Parks and green spaces provide a variety of health benefits by promoting physical activity, connection to nature and opportunities for community engagement. Nature-based programs can even be prescribed by health care providers as part of alternative, cost-effective treatment plans.

>In a paper recently published in the North Carolina Medical Journal, Lincoln Larson and Aaron Hipp, associate professors of parks, recreation and tourism management at NC State, provide an overview of how parks and green spaces can help alleviate contemporary health challenges.

[Source](https://cnr.ncsu.edu/news/2022/04/parks-green-spaces-improve-health/)

## 10.1 Finding the nearest parks to schools in Los Angeles County
You are given information on the point locations of schools in the LA Unified Schoo School District (LAUSD) (``` data/Schools_LAUSD.geojson```) and polygons of LA city parks (```data/Recreation_and_Parks.geojson ```)

1. Load the school and park data. For calcuating distances, what should you check and potentially change? 
2. Find the nearest park to each school in the LAUSD. Capture the  ``` Name ``` of the closest park. Note, this is not as straight forward as just copying and pasting from section-02 week 10! You actually have to think logically what columns are needed (centroids or geometries? what is the target dataframe?). Think carefully! 
3. Create an interactive map which plots the LAUSD schools locations and has a ``` Tooltip ``` capability of showing the ``` FULLNAME ``` of the school (using alias of ``` School Name ```) and the name of the closest park using the alias of ``` Nearest Park Name ```.
4. Save your map as a .html file, upload to your GitHub, and provide a link where anyone can see the final product. Paste this link into the notebook you submit for the assignment. 

## 10.1 SOLUTION

In [10]:
import rasterio
from rasterio.plot import show
from rasterstats import zonal_stats
import osmnx as ox
import geopandas as gpd
import os
import matplotlib.pyplot as plt
%matplotlib inline

base_path = os.path.join("data")
park_path = os.path.join(base_path, "Recreation_and_Parks.geojson")
park = gpd.read_file(park_path)
park.head()

base_path = os.path.join("data")
school_path = os.path.join(base_path, "Schools_LAUSD.geojson")
school = gpd.read_file(school_path)
school.head()

,ID,LOCN,MPD_NAME,ADDRESS,CITY,ZIP,MPD_TYPE,MAP_TYPE,LABEL,MAP_DESC,MPD_DESC,LD,FULLNAME,CDSCODE,CHARTER,MAP_TYPE_MAPPING,TOOLTIP,NLA_URL,geometry
0,1,2835,VISTA DEL VALLE AC DL,12411 N BROMONT AVE,LOS ANGELES,91340,DS,ED,Vista del Valle Ac DL,Spanish Dual Language Program,Dual Language Program,0,Vista del Valle Dual Language Academy Dual Lan...,,,ES,Name: Vista del Valle Dual Language Academy Du...,navigatela/reports/lausd_school_report.cfm?PK=1,POINT (-118.42169 34.29553)
1,2,2820,PARMELEE AVE EL DL SP,1338 E 76TH PL,LOS ANGELES,90001,DS,ED,Parmelee Ave El DL Sp,Spanish Dual Language Program,Dual Language Program,0,Parmelee Avenue Elementary Dual Language Spanish,,,ES,Name: Parmelee Avenue Elementary Dual Language...,navigatela/reports/lausd_school_report.cfm?PK=2,POINT (-118.25099 33.97006)
2,3,2773,OCHOA LC DLC SP,5057 LIVE OAK ST,CUDAHY,90201,DS,ED,Ochoa LC DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Ellen Ochoa Learning Center Dual Language Spanish,,,ES,Name: Ellen Ochoa Learning Center Dual Languag...,navigatela/reports/lausd_school_report.cfm?PK=3,POINT (-118.17660 33.96793)
3,4,2840,WILTON PL EL DLC KO,745 S WILTON PL,LOS ANGELES,90005,DK,ED,Wilton Pl El DLC Ko,Korean Dual Language Program,Dual Language Program,0,Wilton Place Elementary Dual Language Korean,,,ES,Name: Wilton Place Elementary Dual Language Ko...,navigatela/reports/lausd_school_report.cfm?PK=4,POINT (-118.31537 34.05853)
4,5,2814,MILES AVE EL DLC SP,6020 MILES AVE,HUNTINGTON PARK,90255,DS,ED,Miles Ave El DLC Sp,Spanish Dual Language Program,Dual Language Program,0,Miles Avenue Elementary Dual Language Spanish,,,ES,Name: Miles Avenue Elementary Dual Language Sp...,navigatela/reports/lausd_school_report.cfm?PK=5,POINT (-118.21780 33.97755)


In [220]:
park.head()

,OBJECTID,Area_Acres,Park_ID,CD_Real,Address,Region,Name,Park_Class,CD_NEW,TOOLTIP,ACRES,geometry,centroid
0,1,8.433364,512,03,23751 Ingomar St,V,Lazy J Ranch Park,N,12,Name: Lazy J Ranch Park\nLocation: 23751 Ingom...,8.4333635,"POLYGON ((-118.64451 34.21238, -118.64451 34.2...",POINT (-118.64515 34.21312)
1,2,1.387374,883,15,Adjacent to Angels Gate Park at 3400 Gaffey St...,P,Lookout Point Park,N,15,Name: Lookout Point Park\nLocation: Adjacent t...,1.38737377,"POLYGON ((-118.29192 33.71197, -118.29192 33.7...",POINT (-118.29214 33.71246)
2,3,3.173861,320,15,1464 E 109th Street,P,109th Street Recreation Center,N,15,Name: 109th Street Recreation Center\nLocation...,3.17386107,"POLYGON ((-118.24925 33.93590, -118.24917 33.9...",POINT (-118.24859 33.93632)
3,4,0.154871,366,08,4726 8th Avenue,P,Angeles Mesa Park,N,8,Name: Angeles Mesa Park\nLocation: 4726 8th Av...,0.1548711,"POLYGON ((-118.32523 34.00014, -118.32523 34.0...",POINT (-118.32545 34.00021)
4,5,0.960725,213,09,4800 S. Hoover St,P,Julian C. Dixon Park,N,9,Name: Julian C. Dixon Park\nLocation: 4800 S. ...,0.96072542,"POLYGON ((-118.28680 33.99881, -118.28664 33.9...",POINT (-118.28673 33.99919)


In [221]:
from shapely.geometry import Point, MultiPoint
from shapely.ops import nearest_points
import numpy as np

In [222]:
park_pts = MultiPoint([park['geometry'][i].centroid for i in range(len(park))])

In [223]:
nearest_parks = [nearest_points(school["geometry"][i], park_pts) for i in range(len(school))] 

In [224]:
def get_nearest_values(row, other_gdf, point_column='geometry', value_column="geometry"):
    """Find the nearest point and return the corresponding value from specified value column."""
    
    # Create an union of the other GeoDataFrame's geometries:
    other_points = other_gdf["centroid"].unary_union
    
    # Find the nearest points
    nearest_geoms = nearest_points(row[point_column], other_points)
    
    # Get corresponding values from the other df
    nearest_data = other_gdf.loc[other_gdf["centroid"] == nearest_geoms[1]]
    
    nearest_value = nearest_data[value_column].values[0]
    
    return nearest_value

In [225]:
nearest_park_names = [get_nearest_values(school.iloc[i], park, 'geometry', 'Name') for i in range(len(school))]

In [226]:
indices = [park.loc[park['Name'] == nearest_park_names[i]].index[0] for i in range(len(nearest_park_names))]

In [227]:
nps = park.iloc[indices]

In [228]:
import folium
from pyproj import crs
import geopandas as gpd
import matplotlib.pyplot as plt

In [229]:
# 34.0522° N, 118.2437° W
m = folium.Map(location=[34.0522, -118.2437], zoom_start=10, control_scale=True)

for i in range(len(school)):
    folium.Marker(
        location=[school.iloc[i]['geometry'].y, school.iloc[i]['geometry'].x],
        popup=nps.iloc[i]['Name'],
        icon=folium.Icon(color='green', icon='ok-sign'),
    ).add_to(m)

m

In [230]:
outfp = "school_park_map.html"
m.save(outfp)

## you should check that the CRS, and the bounds are the same

In [7]:
park.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [8]:
school.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

## 2